<div style="
    background: linear-gradient(140deg, #0f0f0f 0%, #121212 40%, #1DB954 140%);
    padding: 40px 20px 45px 20px;
    border-radius: 18px;
    text-align: center;
    margin-top: 10px;
    margin-bottom: 25px;
    box-shadow: 0 10px 35px rgba(0,0,0,0.6);
    width: 97%;
"

<img src="logo.png" width="90" style="margin-bottom:18px; filter: drop-shadow(0px 6px 10px rgba(29,185,84,0.6));">
<!-- <img src="logo-spotify.jpg" width="90" style="margin-bottom:18px; filter: drop-shadow(0px 6px 10px rgba(29,185,84,0.6));"> -->


<div style="
    font-size: 2.6em;
    font-weight: 800;
    letter-spacing: 1px;
    color: white;
    margin-bottom: 8px;
">
    Tratamento e Limpeza de Dados
</div>

<div style="
    font-size: 3.2em;
    font-weight: 900;
    color: #1DB954;
    text-shadow: 0px 0px 18px rgba(29,185,84,0.45);
">
    Spotify
</div>

<div style="
    margin-top: 14px;
    color: #b3b3b3;
    font-size: 1.05em;
">
    Ciências de Dados - 3ºH - 2026
</div>

</div>


# Bibliotecas

In [ ]:

# Importar bibliotecas
import pandas as pd
import json
import glob
import matplotlib.pyplot as plt
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
# import spotipy
# from spotipy.oauth2 import SpotifyOAuth

# StreamingHistory

## Bases

<div style="
    background: linear-gradient(145deg, #121212 0%, #181818 60%, #1DB95422 140%);
    padding: 26px 28px;
    border-radius: 16px;
    margin-top: 20px;
    width: 97%;
    color: #EAEAEA;
    box-shadow: 0 8px 22px rgba(0,0,0,0.55);
">

<div style="
    font-size: 1.9em;
    font-weight: 800;
    color: #1DB954;
margin-bottom: 10px;
">
    Arquivo 1 — Dados de Streaming
</div>

<div style="line-height:1.7;">

<div style="margin-bottom:14px;">
    <span style="color:#1DB954; font-weight:700;">Streaming_History_Audio[...]</span><br>
    Histórico de conteúdos em áudio: músicas, podcasts e audiobooks.
</div>

<div style="margin-bottom:14px;">
    <span style="color:#1DB954; font-weight:700;">Streaming_History_Video[...]</span><br>
    Histórico de conteúdos em vídeo: podcasts com vídeo, clipes e conteúdo visual.
</div>

<div style="
    margin-top:18px;
    padding:14px;
    border-radius:10px;
    background:#1DB95418;
    border-left:5px solid #1DB954;
    font-weight:600;
">
    📊 Nesta análise serão utilizados apenas os dados de <b>áudio</b>.
</div>

</div>

</div>


In [ ]:
# Puxando arquivos de áudio 
audio_files = glob.glob("Streaming_History_Audio*.json")

dfs = []

for file in audio_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        df_temp = pd.DataFrame(data)
        df_temp["source_file"] = file 
        dfs.append(df_temp)


# Mesclando áudios 
df_streaming_history = pd.concat(dfs, ignore_index=True)

# Mostrando o dataframe final de áudio
df_streaming_history.head(5)


In [ ]:
df_streaming_history.info()
df_streaming_history.dropna()

## Limpeza de dados

In [ ]:
# Limpeza de dados 
df_streaming_history = df_streaming_history.drop(columns=["ip_addr", "audiobook_title", "audiobook_uri", "audiobook_chapter_uri", "audiobook_chapter_title", "source_file", "offline_timestamp"])

## Tratamento de colunas

In [ ]:
df_streaming_history["ts"] = pd.to_datetime(df_streaming_history["ts"])
df_streaming_history["is_podcast"] = df_streaming_history.apply(
    lambda x: True if pd.notna(x["episode_name"]) else False,
    axis=1
) # Criação de coluna para diferenciar música de podcast


# Renomeando as colunas
column_rename_map = {
    "ts": "played_at",  
    "platform": "platform_used",  
    "ms_played": "milliseconds_played",  
    "conn_country": "connection_country",  

    "master_metadata_track_name": "track_name",  
    "master_metadata_album_artist_name": "artist_name",  
    "master_metadata_album_album_name": "album_name",  
    "spotify_track_uri": "track_uri",  

    "episode_name": "podcast_episode_name",   
    "episode_show_name": "podcast_show_name",  
    "spotify_episode_uri": "podcast_episode_uri",  


    "reason_start": "play_start_reason",  
    "reason_end": "play_end_reason",  

    "shuffle": "shuffle_enabled",  
    "skipped": "skipped_track",  
    "offline": "offline_playback",  
    "offline_timestamp": "offline_timestamp",  

    "incognito_mode": "incognito_mode"
}
df_streaming_history = df_streaming_history.rename(columns=column_rename_map)


## Ajustando o podcast

In [ ]:
podcast_df = df_streaming_history.copy()
df_streaming_history.head(1)

In [ ]:
mask = podcast_df["podcast_episode_uri"].notna()

podcast_df.loc[mask, "track_name"] = podcast_df.loc[mask, "podcast_episode_name"]
podcast_df.loc[mask, "album_name"] = podcast_df.loc[mask, "podcast_show_name"]
podcast_df.loc[mask, "artist_name"] = podcast_df.loc[mask, "podcast_show_name"]
podcast_df.loc[mask, "track_uri"] = podcast_df.loc[mask, "podcast_episode_uri"]

podcast_df = podcast_df.drop(columns=["podcast_episode_name","podcast_show_name", "podcast_episode_uri"])
podcast_df.head(1)

## Normalização dos dados

## Tracks

In [ ]:
unique_tracks = podcast_df[["track_name", "track_uri"]].drop_duplicates()
unique_tracks["id"] = unique_tracks.index
unique_tracks["id"] = unique_tracks["id"]+1
cols = ['id'] + [col for col in unique_tracks.columns if col != 'id']
df_tracks = unique_tracks[cols]
df_tracks = df_tracks.drop_duplicates(subset='track_name', keep='first')
df_tracks = df_tracks.rename(columns={"track_uri":"uri"})
df_tracks.head(2)

## Album

In [ ]:
unique_albuns = podcast_df[["album_name"]].drop_duplicates()
unique_albuns["id"] = unique_albuns.index
unique_albuns["id"] = unique_albuns["id"]+1
cols = ['id'] + [col for col in unique_albuns.columns if col != 'id']
df_albuns = unique_albuns[cols]
df_albuns.head(2)

## Artist

In [ ]:
unique_artists = podcast_df[["artist_name"]].drop_duplicates()
unique_artists["id"] = unique_artists.index
unique_artists["id"] = unique_artists["id"]+1
cols = ['id'] + [col for col in unique_artists.columns if col != 'id']
df_artists = unique_artists[cols]
df_artists.head(2)

## StreamingHistory

In [ ]:
final_streaming_history = podcast_df.copy()

In [ ]:
final_streaming_history = final_streaming_history.merge(df_artists[['artist_name', 'id']], on='artist_name', how='left')
final_streaming_history = final_streaming_history.rename(columns={'id': 'artist_id'})

final_streaming_history = final_streaming_history.merge(df_tracks[['track_name', 'id']], on='track_name', how='left')
final_streaming_history = final_streaming_history.rename(columns={'id': 'track_id'})

final_streaming_history = final_streaming_history.merge(df_albuns[['album_name', 'id']], on='album_name', how='left')
final_streaming_history = final_streaming_history.rename(columns={'id': 'album_id'})
final_streaming_history.head(1)

In [ ]:
final_streaming_history = final_streaming_history.drop(columns=["track_name", "track_uri", "artist_name", "album_name"])
final_streaming_history.head(1)

# LibraryTracks

In [ ]:
with open("YourLibrary.json", "r", encoding="utf-8") as f:
    library = json.load(f)

df_library_tracks = pd.json_normalize(library, record_path=['tracks'], sep='_')
df_library_tracks["liked"] = True
df_library_tracks.head()

In [ ]:
with open('Playlist1.json', 'r', encoding='utf-8') as f:
    playlist_json = json.load(f)

playlists = []
for i in playlist_json["playlists"]:
    playlist = pd.json_normalize(i["items"],sep='_')
    playlist["playlists_lastModifiedDate"] = i["lastModifiedDate"]
    playlist["playlists_name"] = i["name"]
    playlists.append(playlist)

df_playlist = pd.concat(playlists)
df_playlist = df_playlist.rename(columns={"track_trackName": "track", "track_artistName": 'artist', "track_albumName":"album", "track_trackUri":"uri"})
df_playlist = df_playlist[["track", "artist", "album", "uri"]]
df_playlist["in_playlist"] = True
df_playlist.head()

In [ ]:
def nan_to_false(column):
    new_column = []
    for i in column:
        if str(i) == "nan":
            i = False
        new_column.append(i)
    return new_column

In [35]:
library_df = pd.concat([df_library_tracks, df_playlist])
library_df["liked"] = nan_to_false(library_df["liked"])
library_df["in_playlist"] = nan_to_false(library_df["in_playlist"])
library_df.head(1)

,artist,album,track,uri,liked,in_playlist
0,Aaron Smith,Dancin (feat. Luvli),Dancin (feat. Luvli) - Krono Remix,spotify:track:6WkJ2OK163XXS2oARUC9JM,True,False


In [33]:
final_library_df = library_df.copy()

In [36]:
final_library_df = pd.merge(final_library_df, df_artists[['artist_name', 'id']], left_on="artist", right_on='artist_name', how='left')
final_library_df = final_library_df.rename(columns={'id': 'artist_id'})

final_library_df = pd.merge(final_library_df, df_tracks[['track_name', 'id']], left_on="track", right_on='track_name', how='left')
final_library_df = final_library_df.rename(columns={'id': 'track_id'})

final_library_df = pd.merge(final_library_df, df_albuns[['album_name', 'id']], left_on="album", right_on='album_name', how='left')
final_library_df = final_library_df.rename(columns={'id': 'album_id'})
final_library_df = final_library_df[["track_id", "artist_id", "album_id", "in_playlist", "liked"]]
final_library_df.head(1)

,track_id,artist_id,album_id,in_playlist,liked
0,2435.0,2435.0,2435.0,False,True


# SoundCapsule

# Enviar dados para o banco 

In [ ]:
# Carregar variáveis do .env
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")

# Criar string de conexão
DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Criar engine
engine = create_engine(DATABASE_URL)

In [ ]:
username = "Mayumi Castiglioni"

# query = text("""
#     SELECT user_id 
#     FROM "User" 
#     WHERE username = :username
# """)

# with engine.connect() as connection:
#     result = connection.execute(query, {"username": username})
#     row = result.fetchone()

# if row is None:
#     raise ValueError("Usuário não encontrado no banco.")

# user_id = row[0]

user_id = 1

print("User ID encontrado:", user_id)

In [ ]:
# Adicionar a coluna no df 
final_streaming_history["user_id"] = user_id
final_streaming_history = final_streaming_history.drop_duplicates(subset=['played_at'], keep='first')
final_streaming_history

In [ ]:
final_library_df["user_id"] = user_id
final_library_df = final_library_df.dropna(subset=["track_id"])
final_library_df = final_library_df.drop_duplicates(subset=["track_id", "user_id"], keep="first")

In [ ]:
# Rode caso de tudo errado e tenha q dar rollback
from sqlalchemy import text

# This clears the "stuck" transaction
with engine.connect() as conn:
    conn.execute(text("ROLLBACK"))

In [ ]:
df_tracks.to_sql( 
    name="Tracks",
    con=engine,
    if_exists="append",
    index=False
)

df_artists.to_sql(
    name="Artist",
    con=engine,
    if_exists="append",
    index=False
)

df_albuns.to_sql(
    name="Album",
    con=engine,
    if_exists="append",
    index=False
)

In [ ]:
final_streaming_history.to_sql(
    name="StreamingHistory",
    con=engine,
    if_exists="append",
    index=False
)

In [ ]:
final_library_df.to_sql( 
    name="LibraryTracks",
    con=engine,
    if_exists="append",
    index=False
)

<div style="
    background: linear-gradient(145deg, #121212 0%, #181818 60%, #1DB95422 140%);
    padding: 26px 28px;
    border-radius: 16px;
    margin-top: 20px;
    width: 97%;
    color: #EAEAEA;
    box-shadow: 0 8px 22px rgba(0,0,0,0.55);
">

<div style="
    font-size: 1.9em;
    font-weight: 800;
    color: #1DB954;
    margin-bottom: 16px;
">
    Arquivo 2 — Dados da Conta
</div>

<!-- Wrapped -->
---
<div style="margin-bottom:28px;">
    <div style="font-size:1.35em; font-weight:700; color:#FFFFFF;">
        Wrapped2025.json — Métricas Anuais Agregadas
</div>

<div style="margin-top:10px; line-height:1.7;">
    Estatísticas consolidadas do Spotify Wrapped 2025 representando o comportamento musical ao longo do ano.
</div>

</div>

--- 

<!-- Library -->
<div style="margin-bottom:28px;">
<div style="font-size:1.35em; font-weight:700; color:#FFFFFF;">
    YourLibrary.json — Biblioteca Pessoal
</div>

<div style="margin-top:10px; line-height:1.7;">
    Representa o conteúdo salvo pelo usuário: músicas, álbuns e podcasts.
    Diferente do histórico, reflete preferência declarada e não necessariamente consumo real.
</div>

</div>

--- 

<!-- Capsule -->
<div>
<div style="font-size:1.35em; font-weight:700; color:#FFFFFF;">
    YourSoundCapsule.json — Resumo Diário
</div>

<div style="margin-top:10px; line-height:1.7;">
    Estatísticas diárias de consumo musical: quantidade de streams,
    tempo ouvido e ranking diário de músicas e artistas.
</div>



</div>


### Wrapped

In [ ]:
# Carregando Wrapped2025.json
with open("Wrapped2025.json", "r", encoding="utf-8") as f:
    wrapped = json.load(f)

wrapped.keys()

# fazer um df com wrapped 
df_wrapped = pd.DataFrame(wrapped)
df_wrapped.info()


In [32]:
# Tratar dados do wrapped